In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/nlp-mental-health-conversations")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
data=pd.read_csv(path+"/train.csv")
data

## Step 1: Basic EDA

In [ ]:
data.isnull().sum()  ## Check for missing values

In [ ]:
data[data['Response'].isnull()]['Response']

In [ ]:
## Drop the rows with missing information

data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)

In [ ]:
data.info()

## Step 2: Clean the text (Data Preprocessing)

In [ ]:
data['Response'].count()  ## After removal of missing rows

In [ ]:
duplicates = data[data['Response'].duplicated(keep=False)]['Response']  ## Finding the duplicates
duplicates

In [ ]:
duplicates.duplicated().value_counts()

Note: This first selects all occurrences of duplicate values.

Then:

False = first occurrence of each duplicated value

True = every later occurrence of a duplicate value

In [ ]:
## Other way to check Unique values

unique_txt = data['Response'].value_counts()[lambda x: x > 1].index.tolist()
len(unique_txt)  ## Unique value count

In [ ]:
### Code to count the number of punctuation marks

import string
from collections import Counter
from typing import List, Dict

def count_punctuation_in_list(text_list: List[str]) -> Dict[str, int]:
    """
    Analyzes a list of text strings (e.g., cleaned 'Response' data)
    to count the occurrences of each unique punctuation mark.

    Args:
        text_list: A list of strings (your DataFrame column converted to a list).

    Returns:
        A dictionary where keys are punctuation marks (e.g., '.', ',') and
        values are their total counts across all texts.
    """
    # Use collections.Counter for efficient counting
    punctuation_counts = Counter()

    # Define the set of punctuation characters to track
    # string.punctuation includes: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    all_punctuation = string.punctuation

    # Iterate through each text and count punctuation
    for text in text_list:
        if isinstance(text, str):
            for char in text:
                if char in all_punctuation:
                    punctuation_counts[char] += 1

    # Convert the Counter object to a standard dictionary for return
    return dict(punctuation_counts)


counts = count_punctuation_in_list(unique_txt)

print("--- Punctuation Count Results (for EDA) ---")
print(f"Total Unique Punctuation Marks Found: {len(counts)}")
print("\nIndividual Counts:")
# Print the results sorted by count (most frequent first)
for char, count in sorted(counts.items(), key=lambda item: item[1], reverse=True):
    print(f"'{char}': {count}")

# --- EDA Insight ---
# For your assignment, you would use this data to calculate the percentage of
# each punctuation mark relative to the total number of words (for class imbalance analysis).
total_words = sum(len(s.split()) for s in unique_txt)
print(f"\nTotal words in example data: {total_words}")
print(f"Percentage of words followed by a PERIOD (.): {counts.get('.', 0) / total_words * 100:.2f}%")


In [ ]:
import re

def clean_text(u_text):
    for i, txt in enumerate(u_text):
        txt = txt.strip()

        # Collapse multiple spaces
        txt = re.sub(r'\s+', ' ', txt)

        # Remove basic HTML entities (e.g., &amp;, &gt;)
        txt = re.sub(r'&[a-z]+;', '', txt)

        # Replace curly quotes with straight quotes
        txt = re.sub(r'[\u201c\u201d]', '"', txt)
        txt = re.sub(r'[\u2018\u2019]', "'", txt)

        # Replace multiple periods with ellipsis
        txt = re.sub(r'\.{2,}', '...', txt)

        u_text[i] = txt

    return u_text


In [ ]:
clean_unique_txt=clean_text(unique_txt)
clean_unique_txt

In [ ]:
data['Response'].count()

In [ ]:
data['Response'] = data['Response'].drop_duplicates().reset_index(drop=True)

In [ ]:
data['Response'].count()

In [ ]:
# Remove the spaces

data['Response'] = data['Response'].str.strip()
data['Response'] = data['Response'].str.replace(r'\s+', ' ', regex=True)

# Remove basic HTML entities or common artifacts (e.g., &amp;, &gt;)
data['Response'] = data['Response'].str.replace(r'&[a-z]+;', '', regex=True)

# Replace curly quotes with straight quotes
data['Response'] = data['Response'].str.replace(r'[\u201c\u201d]', '"', regex=True)
data['Response'] = data['Response'].str.replace(r'[\u2018\u2019]', "'", regex=True)

# Replace multiple periods (ellipsis) with three periods to standardize
data['Response'] = data['Response'].str.replace(r'\.{2,}', '...', regex=True)

In [ ]:
# Normalize the text

def normalize_text(text):
    # replace punctuation with " <punct> " so spacing is preserved
    text = re.sub(r"([.,!?;:])", r" \1 ", text)
    text = re.sub(r"\s+", " ", text).strip()  # normalize spaces
    # remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    return text.lower()


In [ ]:
final_data=data['Response']
final_data.dropna(inplace=True)
final_data.reset_index(drop=True,inplace=True)
final_data

## Step 3: Creating Synthetic Dataset

In [ ]:
df = pd.DataFrame({
    "input_text": final_data.apply(normalize_text),   # text without punctuation
    "output_text": final_data                             # original text with punctuation
})

In [ ]:
df = df[df["input_text"] != df["output_text"]]
df.reset_index(drop=True,inplace=True)
df

## Step 4: Build Sequence Labeling Dataset for BERT

In [ ]:
# import pandas as pd
from transformers import AutoTokenizer
# import re
from collections import Counter
from typing import List, Dict, Any, Union

# --- Configuration ---
# 0 is the ID for the 'NONE' label (no punctuation)
PUNCT_MAP = {'.': 1, ',': 2, '?': 3, '!': 4, ':': 5, ';': 6}
IGNORE_IDX = -100
BERT_MODEL = 'bert-base-uncased'

# Initialize the tokenizer globally
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)

In [ ]:
def create_labels(punctuated_text: str, tokenized_input: Dict[str, Any]) -> List[int]:
    """
    Creates the final numerical labels aligned with BERT tokens.
    This function relies on the fully punctuated text (the ground truth)
    to extract the punctuation associated with each word.

    Args:
        punctuated_text: The fully punctuated ground-truth sentence (e.g., from 'Response').
        tokenized_input: The output dictionary from the BERT tokenizer.

    Returns:
        A list of integer label IDs, aligned to the BERT tokens.
    """
    # 1. Clean and split the ground truth text into words (including trailing punctuation)
    # We use regex to preserve contractions but split punctuation from words (e.g., ["boy", "."])
    # However, since we rely on word_ids(), a simple split and re.sub is more reliable here.

    # Simple split of the ground truth text
    # Note: If your original data cleaning step was imperfect, this might introduce errors.
    words_with_punct = re.findall(r"[\w']+|[.,?!:;]", punctuated_text.lower())

    # Filter out remaining single-punctuation tokens that are not needed as labels
    # We only care about the punctuation if it's ATTACHED to a word.

    # 2. Extract words and their trailing punctuation label
    word_labels = [] # Stores (word_string_cleaned, label_id)
    current_word = []

    # Iterate through the pre-split tokens to pair words with their trailing punctuation
    i = 0
    while i < len(words_with_punct):
        token = words_with_punct[i]

        # Check if the token is a punctuation mark we care about
        if token in PUNCT_MAP:
            # Punctuation follows the previous word
            if current_word:
                word_labels.append((" ".join(current_word), PUNCT_MAP[token]))
                current_word = []
            # If punctuation starts the sentence (should be cleaned already, but safer to skip)
            else:
                pass

        # If the token is a word
        else:
            # If current_word is not empty, the previous word ended without a target punctuation mark
            if current_word:
                word_labels.append((" ".join(current_word), 0)) # Label 0 = NONE
                current_word = [] # Reset for the new word

            current_word.append(token)

        i += 1

    # Handle the very last word if it was not followed by punctuation
    if current_word:
        word_labels.append((" ".join(current_word), 0)) # Label 0 = NONE


    # --- 3. Align Labels to BERT Tokens using word_ids ---

    # The word_ids() map returns a list of indices where each index corresponds
    # to the original word list (word_labels). None indicates special tokens.
    word_ids = tokenized_input.word_ids()
    labels = []
    previous_word_idx = None

    for word_idx in word_ids:
        # a) Handle Special Tokens and Padding
        if word_idx is None:
            labels.append(IGNORE_IDX)

        # b) Handle First Sub-word Piece of a Word
        elif word_idx != previous_word_idx:
            # Use the index from word_ids to look up the label from our pre-processed list
            if word_idx < len(word_labels):
                # The label is the second element of the tuple (word_string_cleaned, label_id)
                label_id = word_labels[word_idx][1]
                labels.append(label_id)
            else:
                # Should not happen if data is clean, but assign NONE as fallback
                labels.append(0)

        # c) Handle Subsequent Sub-word Pieces (e.g., '##ing')
        else:
            labels.append(IGNORE_IDX)

        previous_word_idx = word_idx

    return labels

In [ ]:
def prepare_dataset(df: pd.DataFrame, max_length: int = 128) -> Dict[str, List[List[int]]]:
    """
    Prepares the entire DataFrame for BERT Sequence Labeling training.

    NOTE: The DataFrame 'df' MUST have two columns:
          - 'input_text': The unpunctuated text (for tokenization).
          - 'output_text': The fully punctuated ground truth text (for label extraction).
    """
    input_ids, attention_masks, labels_list = [], [], []

    # Use tqdm for progress tracking in a real environment
    for index, row in df.iterrows():
        # 1. Tokenize the UNPUNCTUATED input text
        # BERT handles the tokenization and adds CLS/SEP tokens automatically
        tokenized_input = tokenizer(
            row['input_text'],
            truncation=True,
            max_length=max_length,
            return_tensors=None, # Ensure non-tensor output for list processing
            is_split_into_words=False # We feed strings, BERT handles the split
        )

        # 2. Create the labels using the fully PUNCTUATED output text
        # This function performs the alignment
        labels = create_labels(row['output_text'], tokenized_input)

        # Ensure labels length matches input_ids length (critical)
        if len(tokenized_input['input_ids']) == len(labels):
            input_ids.append(tokenized_input['input_ids'])
            attention_masks.append(tokenized_input['attention_mask'])
            labels_list.append(labels)
        else:
            # Log or handle sentences where the token/label alignment failed
            print(f"Skipping sentence {index}: Label length ({len(labels)}) does not match token length ({len(tokenized_input['input_ids'])}).")

    return {'input_ids': input_ids, 'attention_mask': attention_masks, 'labels': labels_list}


## Step 5: Prepare the train-val-test dataset

In [ ]:
def split_processed_data_fixed(processed_data, train_size=0.8, val_size=0.1, test_size=0.1):
    from sklearn.model_selection import train_test_split
    import numpy as np
    if not np.isclose(train_size + val_size + test_size, 1.0):
        raise ValueError("Sizes must sum to 1.0")

    n = len(processed_data['input_ids'])
    indices = np.arange(n)

    temp_size = val_size + test_size
    train_idx, temp_idx = train_test_split(indices, test_size=temp_size, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=test_size/(val_size+test_size), random_state=42)

    return {
        'train': {k: [v[i] for i in train_idx] for k, v in processed_data.items()},
        'validation': {k: [v[i] for i in val_idx] for k, v in processed_data.items()},
        'test': {k: [v[i] for i in test_idx] for k, v in processed_data.items()}
    }

processed_data = prepare_dataset(df,max_length=476)
splits = split_processed_data_fixed(processed_data)
print(f"Training: {len(splits['train']['input_ids'])}")
print(f"Validation: {len(splits['validation']['input_ids'])}")
print(f"Test: {len(splits['test']['input_ids'])}")

## Training Setup

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertForTokenClassification, TrainingArguments, Trainer, AutoTokenizer, DataCollatorForTokenClassification
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import numpy as np


# --- Configuration (MUST MATCH data_processor.py) ---
BERT_MODEL = 'bert-base-uncased'
NUM_LABELS = 7 # NONE(0), PERIOD(1), COMMA(2), QUESTION_MARK(3), EXCLAMATION_MARK(4), COLON(5), SEMICOLON(6)
IGNORE_IDX = -100 # Used for [CLS], [SEP], [PAD], and sub-word tokens
#MAX_LENGTH=476 determined from your EDA


In [ ]:
# --- Step 1: Custom Dataset Class ---
class PunctuationDataset(Dataset):
    """A custom dataset class for PyTorch, compatible with Hugging Face Trainer."""
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Convert list of lists to PyTorch tensors for a single sample
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self,):
        return len(self.encodings['input_ids'])

# --- Step 2: Compute Class Weights for Loss Function ---
def compute_class_weights(train_labels: List[List[int]]) -> torch.Tensor:
    """Calculates inverse frequency weights for handling class imbalance."""
    
    # Flatten the labels and remove IGNORE_IDX
    flat_labels = [label for sublist in train_labels for label in sublist if label != IGNORE_IDX]
    
    # Count frequency of each label
    label_counts = Counter(flat_labels)
    
    # Ensure all labels up to NUM_LABELS are present (even if count is 0)
    for i in range(NUM_LABELS):
        if i not in label_counts:
            label_counts[i] = 1 # Assign 1 to prevent division by zero, though unlikely
            
    total_samples = len(flat_labels)
    
    
    # Calculate weights: Inverse frequency
    # Weight_i = total_samples / (NUM_LABELS * count_i)
    weights = []
    for i in range(NUM_LABELS):
        # Standard inverse frequency scaling
        weight = total_samples / (NUM_LABELS * label_counts[i]) 
        weights.append(weight)

    print("\n--- Class Weight Calculation ---")
    print("Label Frequencies:", {i: label_counts[i] for i in range(NUM_LABELS)})
    print("Calculated Weights (for PyTorch CrossEntropyLoss):")
    
    class_weights = torch.tensor(weights, dtype=torch.float32)
    print(class_weights)
    return class_weights

In [ ]:
# --- Step 3: Custom Trainer for Weighted Loss ---
class WeightedLossTrainer(Trainer):
    """Custom Trainer to override the default loss calculation with class weights."""

    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Store class_weights on CPU initially. It will be moved to the correct device during compute_loss.
        self.class_weights_cpu = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """Custom loss calculation using weighted Cross-Entropy."""
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')

        # Move class_weights to the same device as the logits right before use
        class_weights_on_device = self.class_weights_cpu.to(logits.device)

        # PyTorch CrossEntropyLoss automatically handles the IGNORE_IDX=-100
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_on_device, ignore_index=IGNORE_IDX)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

# --- Step 4: Metric Calculation Function (Macro F1 & Accuracy) ---
def compute_metrics(p: Any) -> Dict[str, float]:
    """Calculates the Macro F1 Score and Accuracy for evaluation."""
    predictions, labels = p.predictions, p.label_ids

    # Get the index of the highest logit (the predicted label ID)
    predicted_label_ids = np.argmax(predictions, axis=2)

    # Flatten arrays and mask out the IGNORE_IDX
    true_labels = labels.flatten()
    pred_labels = predicted_label_ids.flatten()

    mask = true_labels != IGNORE_IDX

    filtered_true = true_labels[mask]
    filtered_pred = pred_labels[mask]

    # Calculate Macro F1 Score (Primary Metric)
    # This treats all classes equally, penalizing the model for missing rare punctuation.
    macro_f1 = f1_score(filtered_true, filtered_pred, average='macro', zero_division=0)

    # Calculate Accuracy (Secondary Metric)
    accuracy = accuracy_score(filtered_true, filtered_pred)

    # Perform EDA on Test Results: Calculate Confusion Matrix (Optional but highly recommended)
    # cm = confusion_matrix(filtered_true, filtered_pred, labels=list(range(NUM_LABELS)))

    return {
        "accuracy": accuracy,
        "macro_f1": macro_f1
        # "confusion_matrix": cm # If you want to log the matrix
    }

In [ ]:
def run_training_setup(splits: Dict[str, Dict[str, List[List[int]]]]):
    """Initializes and runs the training setup."""

    # 1. Prepare Datasets
    train_dataset = PunctuationDataset(splits['train'])
    val_dataset = PunctuationDataset(splits['validation'])
    test_dataset = PunctuationDataset(splits['test']) # Used for final evaluation

    # 2. Compute Weights
    class_weights = compute_class_weights(splits['train']['labels'])

    # 3. Load Model
    # Configure the model to know the number of output labels
    model = BertForTokenClassification.from_pretrained(
        BERT_MODEL,
        num_labels=NUM_LABELS
    )

    # 4. Define Data Collator
    # This collator will handle padding sequences to the same length within each batch
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding='longest')


    # 5. Define Training Arguments
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=5,                     # Recommended initial setting for fine-tuning
        per_device_train_batch_size=8,          # Adjust based on your environment's GPU memory
        per_device_eval_batch_size=8,
        warmup_steps=500,                       # Number of steps for learning rate warmup
        weight_decay=0.01,                      # L2 regularization
        logging_dir='./logs',
        logging_steps=100,
        eval_strategy="epoch",            # Evaluate after each epoch
        save_strategy="epoch",                  # Save checkpoint after each epoch
        load_best_model_at_end=True,            # Load the model with the best validation performance
    )

    # 6. Initialize Trainer with Weighted Loss and Data Collator
    trainer = WeightedLossTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        class_weights=class_weights.to(model.device), # Ensure weights are on the model's device
        data_collator=data_collator, # Pass the data collator here
    )

    # 7. Train the Model (Fine-tuning on Domain Data)
    print("\n--- Starting Model Fine-Tuning ---")
    trainer.train()

    # 8. Final Evaluation on Test Set (for Assignment Report)
    print("\n--- Evaluating Fine-Tuned Model on Test Set ---")
    results = trainer.evaluate(test_dataset)
    print("Fine-Tuned Model Results:", results)

    # --- For Comparison: Baseline Evaluation ---
    # Load the model WITHOUT fine-tuning (Baseline)
    baseline_model = BertForTokenClassification.from_pretrained(BERT_MODEL, num_labels=NUM_LABELS)
    baseline_trainer = Trainer(
        model=baseline_model,
        args=training_args,
        compute_metrics=compute_metrics,
        data_collator=data_collator, # Pass the data collator here as well
        eval_dataset=val_dataset, # Added eval_dataset for baseline_trainer
    )
    print("\n--- Evaluating Pre-Trained Baseline Model on Test Set ---")
    baseline_results = baseline_trainer.evaluate(test_dataset)
    print("Baseline Model Results:", baseline_results)


In [ ]:
run_training_setup(splits)